In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp '/content/drive/MyDrive/Colab Notebooks/Final IMG' -r '/content/'

In [3]:
import os

import cv2
import imutils as imutils
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf # machine learning
from tqdm import tqdm # make your loops show a smart progress meter 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sn

RANDOM_SEED = 1
IMG_SIZE = (256, 256, 3) # size of vgg16 input
IMG_PATH = "/content/Final IMG/"

print(os.listdir(IMG_PATH))

['test', 'train', 'val']


In [4]:
Test_Path = IMG_PATH + "test/"
Training_Path = IMG_PATH + "train/"
Validation_Path = IMG_PATH + "val/"

In [5]:
def number_of_imgs(path):
    print(path)
    for value in os.listdir(path):
        print(value, "has", len(os.listdir(path + value)), "imgs")
    print('\n')
    
    
number_of_imgs(Test_Path)
number_of_imgs(Training_Path)
number_of_imgs(Validation_Path)

/content/Final IMG/test/
caries has 94 imgs
non caries has 22 imgs


/content/Final IMG/train/
caries has 1038 imgs
non caries has 22 imgs


/content/Final IMG/val/
caries has 94 imgs
non caries has 22 imgs




In [ ]:
!pip install rich

In [7]:
from rich.progress import track
#track(os.listdir(Training_Path))

In [8]:
X_train = []
y_train = []
X_val = []
y_val = []

import cv2


for files in track(os.listdir(Training_Path)):
  for file_name in os.listdir(Training_Path + '/' + files):
    img = cv2.imread(Training_Path + '/' + files + '/' + file_name)
    img = cv2.resize(img, (256, 256))
    X_train.append(img)
    y_train.append(files)


for files in os.listdir(Validation_Path):
  for file_name in track(os.listdir(Validation_Path + '/' + files)):
    img = cv2.imread(Validation_Path + '/' + files + '/' + file_name)
    img = cv2.resize(img, (256, 256))
    X_val.append(img)
    y_val.append(files)


Output()

Output()

Output()

In [9]:
X_train= np.array(X_train)
X_val = np.array(X_val)

In [17]:
from sklearn import preprocessing
lb = preprocessing.LabelEncoder()
y_train = lb.fit_transform(y_train)
y_val = lb.transform(y_val)

In [18]:
from imblearn.over_sampling import RandomOverSampler
import numpy as np
import pandas as pd

# Create a synthetic dataset
X = np.array(X_train)
y = np.array(y_train)

# Flatten the images into a 2D representation
X_flattened = X.reshape(X.shape[0], -1)

# Initialize the random oversampler
ros = RandomOverSampler(random_state=0)

# Fit the oversampler on the data
X_resampled, y_resampled = ros.fit_resample(X_flattened, y)

In [19]:
X_resampled.shape

(2076, 196608)

In [20]:
# Reshape the oversampled images back into a 4D representation
X_resampled = X_resampled.reshape(-1, 256, 256, 3)

# Check the class distribution after oversampling
print(np.bincount(y_resampled))

[1038 1038]


In [21]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.applications.vgg16 import VGG16
base = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=IMG_SIZE)

base.trainable=True
model = Sequential()
model.add(base)
model.summary()

58889256/58889256 [==============================] - 4s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [22]:
from tensorflow.keras.layers import Flatten,Dense,Dropout

model.add(Flatten())
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 512)               16777728  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 31,493,442
Trainable params: 31,493,442
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics=['accuracy']
)

In [25]:
X_resampled.shape

(2076, 256, 256, 3)

In [26]:
EPOCHS = 25

history = model.fit(
    x=X_resampled,
    y=y_resampled,
    steps_per_epoch=34,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    validation_steps=25
)

print("Training Done")

Epoch 1/25
34/34 [==============================] - ETA: 0s - loss: 30.9260 - accuracy: 0.6450

34/34 [==============================] - 74s 1s/step - loss: 30.9260 - accuracy: 0.6450 - val_loss: 1.5492 - val_accuracy: 0.2672
Epoch 2/25
34/34 [==============================] - 37s 1s/step - loss: 0.3190 - accuracy: 0.8801
Epoch 3/25
34/34 [==============================] - 37s 1s/step - loss: 0.2152 - accuracy: 0.9340
Epoch 4/25
34/34 [==============================] - 37s 1s/step - loss: 0.1839 - accuracy: 0.9456
Epoch 5/25
34/34 [==============================] - 37s 1s/step - loss: 0.1675 - accuracy: 0.9538
Epoch 6/25
34/34 [==============================] - 37s 1s/step - loss: 0.2760 - accuracy: 0.9432
Epoch 7/25
34/34 [==============================] - 37s 1s/step - loss: 0.1460 - accuracy: 0.9644
Epoch 8/25
34/34 [==============================] - 37s 1s/step - loss: 0.0456 - accuracy: 0.9865
Epoch 9/25
34/34 [==============================] - 37s 1s/step - loss: 0.0123 - accuracy: 0.9966
Epoch 10/25
34/34 [==============================] - 37s 1s/step - loss: 0.0049 - accu

In [27]:
model.save("/content/drive/MyDrive/MODELS/VGG16.h5")

In [28]:
from tensorflow import keras
model_l = keras.models.load_model('/content/drive/MyDrive/MODELS/VGG16.h5')

In [34]:
import numpy as pd 
import cv2

upload_image=r'/content/Final IMG/test/caries/103.png'
image=cv2.imread(upload_image)
converted_img = np.array(image)
img = cv2.resize(converted_img, dsize=(256,256))
img_reshape = np.reshape(img,[1,256,256,3])
y_predict = np.argmax(model_l.predict(img_reshape), axis=1)
# np.argmax(model_l.predict())
y_predict

1/1 [==============================] - 0s 16ms/step


array([0])

In [35]:
model_l.predict(img_reshape)

1/1 [==============================] - 0s 16ms/step


array([[1.000000e+00, 3.151263e-32]], dtype=float32)

In [ ]:
lb.classes_

array(['caries', 'non caries'], dtype='<U10')

In [36]:
X_test = []
y_test = []

for files in os.listdir(Test_Path):
  for file_name in track(os.listdir(Test_Path + '/' + files)):
    img = cv2.imread(Test_Path + '/' + files + '/' + file_name)
    img = cv2.resize(img, (256, 256))
    X_test.append(img)
    y_test.append(files)

Output()

Output()

In [48]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test data
y_pred = np.argmax(model_l.predict(np.array(X_test)), axis=1)

4/4 [==============================] - 1s 206ms/step


In [50]:
# Evaluate the performance of the model using accuracy, precision, recall, and F1 score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0
